In [ ]:
import torch
import numpy as np
import normalization
import kornia
from PIL import Image
import matplotlib.pyplot as plt

def white_balance(image):
    image = image.numpy().copy()
    valid = np.all(~np.isnan(image), axis=2)
    image_valid = image[valid]
    image_valid = image_valid / image_valid.mean(axis=0)
    image_valid = np.clip(image_valid, np.percentile(image_valid, 1), np.percentile(image_valid, 99))
    image_valid = image_valid - np.min(image_valid)
    image_valid = image_valid / np.max(image_valid)
    image[~valid] = 0
    image[valid] = image_valid
    return image

def stretch(image, mask):
    mask = mask.numpy().copy()
    valid = np.all(~np.isnan(mask), axis=2)
    image_valid = image[valid]
    image_valid = np.clip(image_valid, np.percentile(image_valid, 1, axis=0), np.percentile(image_valid, 99, axis=0))
    image_valid = image_valid - np.min(image_valid, axis=0)
    image_valid = image_valid / np.max(image_valid, axis=0)
    mask[~valid] = 0
    mask[valid] = image_valid
    return mask

In [ ]:
J = torch.load('test/20150419T033644.000Z.pt')

In [ ]:
Image.fromarray(
    np.uint8(normalization.tone_map(
        stretch(white_balance(J), J)
    ) * 255)
)

In [ ]:
a=np.uint8(normalization.tone_map(
    stretch(white_balance(J), J)
) * 255)
b=np.uint8(normalization.tone_map(
    white_balance(J)
) * 255)

In [ ]:
plt.hist([a[:, :, 0].flatten(), a[:, :, 1].flatten(), a[:, :, 2].flatten()], color=['r', 'g', 'b'], bins=20)
plt.show()
plt.hist([b[:, :, 0].flatten(), b[:, :, 1].flatten(), b[:, :, 2].flatten()], color=['r', 'g', 'b'], bins=20)
plt.show()

In [ ]:
plt.hist([J[:, :, 0].flatten(), J[:, :, 1].flatten(), J[:, :, 2].flatten()], color=['r', 'g', 'b'], bins=20)
plt.show()

In [ ]:
a=white_balance(J)
plt.hist([a[:, :, 0].flatten(), a[:, :, 1].flatten(), a[:, :, 2].flatten()], color=['r', 'g', 'b'], bins=20)
plt.show()

In [ ]:
lab = kornia.color.rgb_to_lab(kornia.color.linear_rgb_to_rgb(J.movedim(2, 0)))
valid = torch.all(~lab.isnan(), dim=0)
l, a, b = lab[:, valid]

In [ ]:
a = np.zeros_like(l.numpy())
args_l_valid = ~l.isnan()
l_valid = l[args_l_valid]
l_valid = np.clip(l_valid, np.percentile(l_valid, 1), np.percentile(l_valid, 99))
l_valid = l_valid - l_valid.min()
l_valid = l_valid / l_valid.max()
a[args_l_valid] = l_valid

In [ ]:
plt.imshow(a, cmap='gray')

In [ ]:
J = torch.load('test/20150419T033644.000Z.pt')
valid = torch.all(~J.isnan(), dim=2)
J_valid = J[valid].numpy()
# J_valid = J_valid / J_valid.mean(dim=0)
J_valid = np.clip(J_valid, np.percentile(J_valid, 1, axis=0), np.percentile(J_valid, 99, axis=0))
J_valid = J_valid - J_valid.min(axis=0)
J_valid = J_valid / J_valid.max(axis=0)
J[valid] = torch.tensor(J_valid, dtype=torch.float32)
lab = kornia.color.rgb_to_lab(kornia.color.linear_rgb_to_rgb(J.movedim(2, 0)))
l, a, b = lab[:, valid]
avg_a = a.mean()
avg_b = b.mean()
a_shift = -avg_a
b_shift = -avg_b
a_delta = a_shift * (l / 100) * 1.1
b_delta = b_shift * (l / 100) * 1.1
a, b = a + a_delta, b + b_delta
lab[1, valid] = a
lab[2, valid] = b
rgb = kornia.color.lab_to_rgb(lab).movedim(0, 2)

In [ ]:
rgb[~valid] = 0
plt.imshow(rgb.numpy())

In [ ]:
Image.fromarray(np.uint8(rgb.numpy() * 255))